In [2]:
### ignore warning 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning) 

import numpy as np
import pandas as pd
import re
import glob
import spacy 
import nltk
import os
import nltk
from nltk.tokenize import TreebankWordTokenizer ,word_tokenize
from nltk.stem import PorterStemmer , WordNetLemmatizer
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import everygrams

# nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
from nltk import everygrams

# display  max row 
#pd.set_option("display.max_rows", None)
import xlrd
 
## tqdm 
from tqdm._tqdm_notebook import tqdm_notebook,tqdm
tqdm_notebook.pandas()

/home/nick/envDITP/lib/python3.8/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
[nltk_data] Downloading package punkt to /home/nick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/nick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#! pip install xlrd
#df = xlrd.open_workbook("DITP3_09022022_news_test_set_new.xlsx")
df = pd.read_csv('DITP3_09022022_news_test_set_new.xlsx - 09022022_news_test_set.csv')
df_more = pd.read_csv('new_additional_test_set.csv')

In [4]:
df_more.drop_duplicates(subset=['title'], keep='first', inplace=True)

In [5]:
df.columns

Index(['Column1', 'new_id', 'title', 'content', 'Label verified by AJ', 'url',
       'tags', 'publisher', 'published_date'],
      dtype='object')

In [6]:
def prep_data(data) :
    word_domain = []
    wpt = nltk.WordPunctTokenizer()
    stop_word  = stopwords.words('english')
    #data = data_test
    data = data.strip()
    # optional  base on inspection  
    #data  =  data.lower()
    ########################### 
    result = string.punctuation
    ### Clean HTML Tacontents = df_clean.content.to_list()
    data = re.sub(r'\n','',data)
    data = re.sub(r'\xa0','',data)
    data = re.sub(r"\\","",data)
    data = re.sub(r'<li>','',data)
    data = re.sub(r'</li>','',data)
    data = re.sub(r'</ul>','',data)
    data = re.sub(r'<td>','',data)
    data = re.sub(r'</td>','',data)
    
    
    data= re.sub(r'\u200b', '', data)
    data = re.sub(r'[ๆฯ!#$&%\"\'()*+,-./:;<=>?@\[\]\\^_`{}|~]',' ', data)
    data = re.sub(r'\d',' ', data)
    data = re.sub(r'\n', ' ', data)
    #data = re.sub(r'[^a-zA-Z\s]', '', data, re.I|re.A)
    data = re.sub(r'・','',data)
    

   
    lst_pun = [i for i in result] + ['•'] +['“','”','’']+['©']+['£']+['—'] +['-']
    data = data.replace('|',' ')
    data = data.replace('-', ' ')
    data = data.replace('|****|',' ')
    data = wpt.tokenize(data)
    data =  ' '.join( i for i in data if i not in lst_pun and i.isnumeric() == False ) 
    data  =  nltk.tokenize.word_tokenize(data)
    
    data =  [i for i in data if i not in  stop_word ]
    data   = ' '.join(data)
    return data


    #data  = list(everygrams(data, 1, 3))

In [8]:
! python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
import spacy 
from spacy import displacy
nlp1 = spacy.load('en_core_web_sm') #

In [8]:
## Apply NER 
def ner_process(sentens_string) :
    doc1 = nlp1(sentens_string) 
    sentenss = []
    for sr in  doc1 :
        if sr.ent_type_  not in  exclude_ner :
               sentenss.append(str(sr)) 
    return ' '.join(sentenss)            

# x =  ner_process()

In [9]:
def ner_process(sentens_string) :
    doc1 = nlp1(sentens_string) 
    sentenss = []
    for sr in  doc1 :
        if sr.ent_iob_  == 'O' :
               sentenss.append(str(sr)) 
    return ' '.join(sentenss)        

In [46]:
## prep only NOUN 

def noun_ADJ(x):
    x  = x.split()
    pos_comment = nltk.pos_tag(x)
    filtered = [word[0] for word in pos_comment if word[1] in ['NN','JJ','JJR','JJS','NNP','NNS']]
    # to filter both noun and verbs
    #filtered = [word[0] for word in pos_comment if word[1] in ['NN','VB', 'VBD', 'VBG', 'VBN', 'VBZ']]
    return filtered


In [11]:
def bi_gram(data) :
    return list(ngrams(data, 2))

def tri_gram(data):
    return  lsit(ngrams(data,3))

def chang_ngram_form(data_list): 
        lst_bi_gram = []
        for tri in  data_list :
            x,y  =  tri 
            new_xy =  f'{x}_{y}'
            lst_bi_gram.append(new_xy)
        return lst_bi_gram
    
def tfidf(copus):
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, 
    max_features=n_features, stop_words='english', ngram_range=(1, 2))
    tfidf = tfidf_vectorizer.fit_transform(corpus)
    return tfidf    

    

In [26]:
df['content_clean'] =  df['content'].apply(prep_data)
df_more['content_clean'] =df_more['content'].apply(prep_data)

In [27]:
#df['content_clean']  =  df['content'].apply( lambda x: x.lower())
df_more['content_clean']  =  df_more['content'].apply( lambda x: x.lower())

In [28]:
#df['content_clean_ner'] =  df['content_clean'].apply(ner_process)
df_more['content_clean_ner'] =  df_more['content_clean'].apply(ner_process)

In [47]:
#df['content_clean_ner_n'] = df['content_clean_ner'].apply(noun_only)
#df['content_clean_ner_n'] = df['content_clean_ner'].apply(noun_only)
df_more['content_clean_ner_n'] = df_more['content_clean_ner'].apply(noun_ADJ)

In [48]:
df_more['content_clean_ner_n'][0]

['–',
 'api',
 'new',
 'specification',
 'oilfield',
 'hammer',
 'unions',
 'future',
 'interoperability',
 'field',
 'equipment',
 'enhance',
 'safety',
 'natural',
 'gas',
 'oil',
 'industry',
 'workers',
 '’s',
 'specification',
 'spec',
 'hammer',
 'unions',
 'global',
 'standard',
 'incompatibility',
 'hammer',
 'union',
 'components',
 'common',
 'issue',
 'industry',
 'field',
 'incidents',
 'response',
 'industry',
 'calls',
 'heightened',
 'standards',
 'hammer',
 'union',
 'components',
 'interoperability',
 'safety',
 'prevent',
 'potential',
 'failures',
 'api',
 '’s',
 'upstream',
 'standard',
 'mismatching',
 'hammer',
 'unions',
 'vital',
 'step',
 'workers',
 'field',
 'operations',
 'safe',
 '”',
 'senior',
 'vice',
 'president',
 'api',
 '’s',
 'global',
 'industry',
 'services',
 '“',
 'spec',
 'addresses',
 'current',
 'safety',
 'issues',
 'field',
 'environments',
 'enhanced',
 'performance',
 'petroleum',
 'natural',
 'gas',
 'operations',
 'natural',
 'gas',
 'o

In [49]:
#df['content_clean_ner_n']  =df['content_clean_ner_n'].apply(lambda x : [ i.lower() for i in  x] )
df_more['content_clean_ner_n']  =df_more['content_clean_ner_n'].apply(lambda x : [ i.lower() for i in  x] )

In [50]:
df_more

,published_at,title,description,content,url,url_to_image,tags,id,original_path,ingestion_date,published_year,published_date,publisher,content_clean,content_clean_ner,content_clean_ner_n
0,2022-03-03 00:00:00.000,API publishes new safety specification for oil...,NaN,WASHINGTON – The American Petroleum Institute ...,https://www.worldoil.com/news/2022/3/3/api-pub...,NaN,NaN,b51649cb58a3439bfedbe77162300267,wasbs://raw-data@tradeprodstditp.blob.core.win...,2022-03-04,2022,2022-03-03,World Oil,washington – the american petroleum institute ...,– ( api ) published a new specification for ma...,"[–, api, new, specification, oilfield, hammer,..."
1,2022-01-17 00:00:00.000,BP and Oman form renewable energy and hydrogen...,NaN,BP and the Ministry of Energy and Minerals in ...,https://www.worldoil.com/news/2022/1/17/bp-and...,NaN,NaN,e5dcc6d8a18a0ef254a3b9caf4ffe90e,wasbs://raw-data@tradeprodstditp.blob.core.win...,2022-01-18,2022,2022-01-17,World Oil,bp and the ministry of energy and minerals in ...,bp and and minerals in signed a strategic fram...,"[bp, minerals, strategic, framework, agreement..."
3,2021-12-12 00:00:00.000,Iran bases its 2022 budget on $60 oil,NaN,(Bloomberg) --The budget bill for Iran’s next ...,https://www.worldoil.com/news/2021/12/10/iran-...,NaN,NaN,fbe1a6a1078c009c4637039ef1766023,wasbs://raw-data@tradeprodstditp.blob.core.win...,2021-12-13,2021,2021-12-12,World Oil,(bloomberg) --the budget bill for iran’s next ...,( ) --the budget bill for ’s is based on an oi...,"[budget, bill, ’s, oil, price, barrel, semi, o..."
4,2021-02-28 19:00:00.000,Heavy rain dampens Bangladesh rice production,NaN,"DHAKA, BANGLADESH — Heavy rain has impeded Ban...",https://ricenewstoday.com/?p=123841,https://www.world-grain.com/ext/resources/Arti...,NaN,296a6634da82333ae0b72937c6a4b959,wasbs://raw-data@stditp.blob.core.windows.net/...,2021-08-02,2021,2021-02-28,Rice News Today,"dhaka, bangladesh — heavy rain has impeded ban...",", bangladesh — heavy rain has impeded banglade...","[bangladesh, —, heavy, rain, bangladesh, ’s, r..."
5,2021-04-01 19:00:00.000,Thailand rice sector recovering from drought,NaN,"BANGKOK, THAILAND — Over the past two years Th...",https://ricenewstoday.com/?p=123924,https://www.world-grain.com/ext/resources/Arti...,NaN,0e3d958123fc7fd632b20e096fd93554,wasbs://raw-data@stditp.blob.core.windows.net/...,2021-08-02,2021,2021-04-01,Rice News Today,"bangkok, thailand — over the past two years th...",", — over thailand ’s rice production has been ...","[thailand, ’s, rice, production, drought, expe..."
6,2021-07-04 19:00:00.000,Thai rice exports drop 21 pct in six months,NaN,Charoen predicted that Thailand’s total rice e...,https://ricenewstoday.com/?p=124100,https://www.theborneopost.com/newsimages/2020/...,NaN,7d8d14b256921006bf11838b261374dd,wasbs://raw-data@stditp.blob.core.windows.net/...,2021-08-02,2021,2021-07-04,Rice News Today,charoen predicted that thailand’s total rice e...,predicted that ’s total rice export volume wou...,"[’s, total, rice, export, volume, lower, targe..."
9,2021-10-05 05:00:14.000,Analysis: The end of cheap food,NaN,Costs are rising across the board… including c...,https://www.poultrynews.co.uk/business-politic...,https://www.poultrynews.co.uk/wp-content/uploa...,NaN,ae9f209a7ca2dbb83e01c371f8acb1a3,wasbs://raw-data@tradeprodstditp.blob.core.win...,2021-10-06,2021,2021-10-05,Poultry News,costs are rising across the board… including c...,costs are rising across the board … including ...,"[costs, board, carbon, dioxide, gas, feed, pac..."


In [53]:
dict_df  =  pd.read_csv('/home/nick//DITP/trade-midas/ditp3_run_test/GNA/corpus/existing/event_type_keyword.csv', index_col=False)

In [54]:
dict_df.head(50)

,event_type,keyword
0,Economics Finance,Fintech
1,Economics Finance,NPL
2,Economics Finance,Rate Halt
3,Economics Finance,Default Rate
4,Economics Finance,Credit Approval
5,Economics Finance,bank
6,Economics Finance,Macroeconomic
7,Economics Finance,GDP
8,Economics Finance,wage
9,Economics Finance,Household Debt


In [55]:
'Derivative' in  dict_df['keyword'].to_list()

True

In [64]:
#dict_df['keyword_lower'] = dict_df['keyword'].str.lower()

In [56]:
dict_df['event_type'].value_counts().keys()

Index(['Trade Policy', 'Economics Finance', 'Natural Diaster',
       'International Politics', 'Logistic',
       'Market Trend & Consumer Behavior', 'Service', 'Economic Exports'],
      dtype='object')

In [69]:
def find_word_event(data):
        df_key_ls = []
        for i in dict_df['keyword'].to_list():
            if (len(i.split(' ')) > 1) and (i.lower() in  ' '.join(data)) :
                df_key_ls.append(i)
            elif (len(i.split(' ')) == 1) and ( i.lower() in  data)  :
                df_key_ls.append(i)
        return     df_key_ls   
                
            

In [88]:
def find_word1(data):
    
        dicts = dict_df.to_dict(orient='split')
        convert_dict = {}
    
        for i in dicts.get('data') :
            convert_dict[i[-1]] = i[0]
            
            
        df_key_ls = []
        
        for i in dict_df['keyword'].to_list():
            if (len(i.split(' ')) > 1) and (i.lower() in  ' '.join(data)) :
                df_key_ls.append((i,convert_dict.get(i)))
                
            elif (len(i.split(' ')) == 1) and ( i.lower() in  data)  :
                df_key_ls.append((i,convert_dict.get(i)))
        return     df_key_ls   

In [89]:
#df['key_word'] = df.apply( lambda x :find_word(x['content_clean']) , axis=1 )
df_more['key_word'] = df.apply( lambda x :find_word1(x['content_clean']) , axis=1 )
df_more['key_word'][0]

[('Train', 'Logistic'),
 ('Sector', 'Market Trend & Consumer Behavior'),
 ('products', 'Market Trend & Consumer Behavior'),
 ('customer', 'Market Trend & Consumer Behavior'),
 ('business', 'Trade Policy'),
 ('Import', 'Trade Policy'),
 ('business', 'Trade Policy'),
 ('exchange', 'Trade Policy'),
 ('foreign', 'Trade Policy'),
 ('cost', 'Trade Policy')]

In [59]:
df_more['key_word']

0    [Train, Sector, products, customer, business, ...
1    [NAL, Economic, Supply, Demand, Investment, Re...
3    [bank, NAL, Demand, Income, Fed, Legal, law, S...
4    [bank, NAL, Economic, Act, trade, Government, ...
5    [Supply, Demand, trade, Sale, Sector, market, ...
6    [bank, NAL, Economic, Account, Act, Protest, I...
9    [NAL, Stock, Act, trade, Machine, Election, Sh...
Name: key_word, dtype: object

In [80]:
dict_df.set_index('event_type').T.to_dict('list')

/tmp/ipykernel_118827/3478376975.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  dict_df.set_index('event_type').T.to_dict('list')


{'Economics Finance': ['high stockpiles'],
 'International Politics': ['law'],
 'Logistic': ['Fleet'],
 'Market Trend & Consumer Behavior': ['wholesaler'],
 'Natural Diaster': ['damage'],
 'Service': ['travel bubble'],
 'Trade Policy': ['sanction'],
 'Economic Exports': ['Gold exports']}

In [73]:
dicts = dict_df.to_dict(orient='split')
#print((dicts.get('data')))

convert_dict = {}

for i in dicts.get('data') :
    convert_dict[i[-1]] = i[0]
    #print(i)


# def get_tag():
    
#     for i in (dicts.get('data')) :
#         if i[-1] in  

convert_dict
#df['tag'] =  df['key_word'].apply( lambda x : list(set([ convert_dict.get(i)   for i  in  x ]) ))

#df_more['tag'] =  df_more['key_word'].apply( lambda x : list(set([ convert_dict.get(i)   for i  in  x ]) )) 

['Economics Finance', 'Fintech']
['Economics Finance', 'NPL']
['Economics Finance', 'Rate Halt']
['Economics Finance', 'Default Rate']
['Economics Finance', 'Credit Approval']
['Economics Finance', 'bank']
['Economics Finance', 'Macroeconomic']
['Economics Finance', 'GDP']
['Economics Finance', 'wage']
['Economics Finance', 'Household Debt']
['Economics Finance', 'Credit Rating']
['Economics Finance', 'fiscal']
['Economics Finance', 'Fluctuation']
['Economics Finance', 'Foreign Exchange Reserve']
['Economics Finance', 'NAL']
['Economics Finance', 'credibility']
['Economics Finance', 'Inward Foreign Direct Investment']
['Economics Finance', 'Nasdaq']
['Economics Finance', 'Foreign Direct Investment']
['Economics Finance', 'IFDI']
['Economics Finance', 'Rebound']
['Economics Finance', 'OFDI']
['Economics Finance', 'IMF']
['Economics Finance', 'Labour']
['Economics Finance', 'Sterilization']
['Economics Finance', 'Economist']
['Economics Finance', 'Depreciate']
['Economics Finance', 'Econ

{}

In [41]:
df_more.shape

(7, 18)

In [42]:
df_tag = pd.DataFrame( columns  = dict_df['event_type'].value_counts().keys(),  index =  range(0,7))

In [38]:
ee = df['tag'].to_list()
# for u in ee :
#     print(u)

KeyError: 'tag'

In [43]:
for col in  dict_df['event_type'].value_counts().keys() :
    count = 0 
    for jj in  df_more['tag'].to_list() :
        for jjj in jj :
            print(jjj)
            print(jjj, col)
            if jjj == col : 
                df_tag.loc[count,jjj] =  1
        count +=1 

Logistic
Logistic Trade Policy
Trade Policy
Trade Policy Trade Policy
Market Trend & Consumer Behavior
Market Trend & Consumer Behavior Trade Policy
Logistic
Logistic Trade Policy
Trade Policy
Trade Policy Trade Policy
Economics Finance
Economics Finance Trade Policy
Natural Diaster
Natural Diaster Trade Policy
Market Trend & Consumer Behavior
Market Trend & Consumer Behavior Trade Policy
Service
Service Trade Policy
International Politics
International Politics Trade Policy
Trade Policy
Trade Policy Trade Policy
Economics Finance
Economics Finance Trade Policy
Market Trend & Consumer Behavior
Market Trend & Consumer Behavior Trade Policy
International Politics
International Politics Trade Policy
Logistic
Logistic Trade Policy
Trade Policy
Trade Policy Trade Policy
Economics Finance
Economics Finance Trade Policy
Natural Diaster
Natural Diaster Trade Policy
International Politics
International Politics Trade Policy
Trade Policy
Trade Policy Trade Policy
Economics Finance
Economics Fina

In [44]:
df_tag

,Trade Policy,Economics Finance,Natural Diaster,International Politics,Logistic,Market Trend & Consumer Behavior,Service,Economic Exports
0,1,NaN,NaN,NaN,1,1,NaN,NaN
1,1,1,1,1,1,1,1,NaN
2,1,1,NaN,1,NaN,1,NaN,NaN
3,1,1,1,1,1,NaN,NaN,NaN
4,1,1,NaN,NaN,NaN,1,1,NaN
5,1,1,1,1,1,1,1,NaN
6,1,1,1,1,1,1,1,NaN


In [45]:
df_tag.replace(np.nan,0,inplace=True)

In [57]:
df_more.reset_index( inplace=True)

In [58]:
df_all = pd.concat([df_more, df_tag],axis=1)

In [59]:
df_all.columns
df_all['Label verified by AJ'] = ''

In [64]:
sd = ['id', 'title', 'content', 'Label verified by AJ' , 'url','tags', 'publisher', 'published_date', 'content_clean'
     , 'key_word', 'tag',
       'Trade Policy', 'Economics Finance', 'Natural Diaster',
       'International Politics', 'Logistic',
       'Market Trend & Consumer Behavior', 'Service', 'Economic Exports'
     ]

In [65]:
df_all1 =df_all[sd]

In [66]:
df_all1.to_csv('added_news_dataset.csv')

In [ ]:
# Index(['Column1', 'new_id', 'title', 'content', 'Label verified by AJ', 'url',
#        'tags', 'publisher', 'published_date', 'content_clean'],
#       dtype='object')

In [222]:
df_all.to_excel('20220218_news_test_set_category_tag.xlsx')

In [50]:
ddd = pd.read_excel('20220218_news_test_set_category_tag.xlsx')

XLRDError: Excel xlsx file; not supported

In [4]:
! pip install google_spreadsheet
! pip install google-auth-oauthlib

  ERROR: Command errored out with exit status 1:
   command: /home/nick/envDITP/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9pqrehdd/google-spreadsheet/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9pqrehdd/google-spreadsheet/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-htjt9_wa
       cwd: /tmp/pip-install-9pqrehdd/google-spreadsheet/
  Complete output (6 lines):
  usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: setup.py --help [cmd1 cmd2 ...]
     or: setup.py --help-commands
     or: setup.py cmd --help
  
  error: invalid command 'bdist_wheel'
  ----------------------------------------
  ERROR: Failed building wheel for google-spreadsheet
  Running setup.py clean for google-spreadsheet
Failed to build google-spreadsheet
    Running setup.py install f

In [3]:
## access g-drive 

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()

drive = GoogleDrive(gauth)

InvalidConfigError: Invalid client secrets file ('Error opening file', 'client_secrets.json', 'No such file or directory', 2)